In [2]:


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
from sklearn.model_selection import train_test_split


BATCH_SIZE = 64
IMG_SIZE = 224
AUTOTUNE = tf.data.AUTOTUNE
NUM_CLASSES = 10
SEED = 42


(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
y_train = y_train.flatten()
y_test = y_test.flatten()


x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.1, random_state=SEED, stratify=y_train)


def preprocess_image(image, label, training):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32)
    if training:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_brightness(image, max_delta=0.1)
        image = tf.image.random_contrast(image, 0.9, 1.1)
    return image, label

def make_dataset(images, labels, training=False):
    ds = tf.data.Dataset.from_tensor_slices((images, labels))
    if training:
        ds = ds.shuffle(10000, seed=SEED)
    ds = ds.map(lambda x,y: preprocess_image(x,y, training), num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

train_ds = make_dataset(x_train, y_train, training=True)
val_ds = make_dataset(x_val, y_val, training=False)
test_ds = make_dataset(x_test, y_test, training=False)


base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    weights='imagenet'
)
base_model.trainable = False  # نبدأ بتجميد

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = tf.keras.applications.efficientnet.preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='swish')(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = keras.Model(inputs, outputs)

# ------------ تجميع النموذج ----------------
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ------------ Callbacks --------------------
checkpoint_path = "best_model.h5"
callbacks = [
    keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1),
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, mode='max', verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

# ------------ مرحلة 1: تدريب الطبقات العليا فقط ------------
EPOCHS_STAGE_1 = 8
history1 = model.fit(
    train_ds,
    epochs=EPOCHS_STAGE_1,
    validation_data=val_ds,
    callbacks=callbacks
)

# ------------ مرحلة 2: Fine-Tuning أعمق ----------------
base_model.trainable = True
fine_tune_at = int(len(base_model.layers) * 0.2)  # جمد أول 20% فقط (80% مفتوحة)
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

EPOCHS_STAGE_2 = 30
history2 = model.fit(
    train_ds,
    epochs=EPOCHS_STAGE_2,
    validation_data=val_ds,
    callbacks=callbacks
)

# ------------ تقييم على مجموعة الاختبار ----------------
print("Loading best weights from checkpoint...")
if os.path.exists(checkpoint_path):
    model.load_weights(checkpoint_path)
test_loss, test_acc = model.evaluate(test_ds)
print(f"✅ Test accuracy: {test_acc*100:.2f}%")

# ------------ حفظ النموذج النهائي ----------------
model.save("cifar10_efficientnet_finetuned.h5")
print("Model saved to cifar10_efficientnet_finetuned.h5")


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,380,077 (16.71 MB)

 Trainable params: 330,506 (1.26 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

...

704/704 ━━━━━━━━━━━━━━━━━━━━ 141s 172ms/step - accuracy: 0.9935 - loss: 0.0180 - val_accuracy: 0.9724 - val_loss: 0.1039 - learning_rate: 5.0000e-05
Epoch 14/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.9957 - loss: 0.0144
Epoch 14: val_accuracy improved from 0.97240 to 0.97480, saving model to best_model.h5


704/704 ━━━━━━━━━━━━━━━━━━━━ 120s 171ms/step - accuracy: 0.9957 - loss: 0.0143 - val_accuracy: 0.9748 - val_loss: 0.0968 - learning_rate: 5.0000e-05
Epoch 15/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.9966 - loss: 0.0105
Epoch 15: val_accuracy did not improve from 0.97480
704/704 ━━━━━━━━━━━━━━━━━━━━ 120s 170ms/step - accuracy: 0.9966 - loss: 0.0105 - val_accuracy: 0.9738 - val_loss: 0.1024 - learning_rate: 5.0000e-05
Epoch 16/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.9962 - loss: 0.0114
Epoch 16: val_accuracy did not improve from 0.97480
704/704 ━━━━━━━━━━━━━━━━━━━━ 142s 170ms/step - accuracy: 0.9962 - loss: 0.0114 - val_accuracy: 0.9732 - val_loss: 0.1047 - learning_rate: 5.0000e-05
Epoch 17/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.9965 - loss: 0.0109
Epoch 17: val_accuracy did not improve from 0.97480

Epoch 17: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
704/704 ━━━━━━━━━━━━━━━━━━━━ 142s 170ms/step - accur


Epoch 23: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-06.
704/704 ━━━━━━━━━━━━━━━━━━━━ 143s 171ms/step - accuracy: 0.9984 - loss: 0.0054 - val_accuracy: 0.9752 - val_loss: 0.1001 - learning_rate: 1.2500e-05
Epoch 24/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.9985 - loss: 0.0054
Epoch 24: val_accuracy improved from 0.97520 to 0.97560, saving model to best_model.h5


704/704 ━━━━━━━━━━━━━━━━━━━━ 142s 171ms/step - accuracy: 0.9985 - loss: 0.0054 - val_accuracy: 0.9756 - val_loss: 0.1004 - learning_rate: 6.2500e-06
Epoch 25/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.9984 - loss: 0.0053
Epoch 25: val_accuracy did not improve from 0.97560
704/704 ━━━━━━━━━━━━━━━━━━━━ 141s 170ms/step - accuracy: 0.9984 - loss: 0.0053 - val_accuracy: 0.9750 - val_loss: 0.1007 - learning_rate: 6.2500e-06
Epoch 26/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.9983 - loss: 0.0052
Epoch 26: val_accuracy did not improve from 0.97560

Epoch 26: ReduceLROnPlateau reducing learning rate to 3.12499992105586e-06.
704/704 ━━━━━━━━━━━━━━━━━━━━ 119s 169ms/step - accuracy: 0.9983 - loss: 0.0052 - val_accuracy: 0.9750 - val_loss: 0.1017 - learning_rate: 6.2500e-06
Epoch 27/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.9983 - loss: 0.0047
Epoch 27: val_accuracy did not improve from 0.97560
704/704 ━━━━━━━━━━━━━━━━━━━━ 119s 169ms/step - accura

✅ Test accuracy: 97.27%
Model saved to cifar10_efficientnet_finetuned.h5


In [27]:
print(f"{test_acc*100:.2f}%")

97.27%


In [29]:
from google.colab import files
from PIL import Image
import numpy as np
import tensorflow as tf

# رفع الصورة من جهازك
uploaded = files.upload()  # حيطلب منك اختيار الصورة

# نفترض رفع صورة واحدة فقط
file_path = list(uploaded.keys())[0]

# تحميل النموذج
model = tf.keras.models.load_model("cifar10_efficientnet_finetuned.h5")

# قائمة الفئات CIFAR-10
class_names = ["airplane", "automobile", "bird", "cat", "deer",
               "dog", "frog", "horse", "ship", "truck"]

# تحميل الصورة وتحويلها للحجم المطلوب
img = Image.open(file_path).convert("RGB")
img = img.resize((224, 224))
img_array = np.array(img).astype("float32")

# تطبيع باستخدام EfficientNet preprocess
img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
img_array = np.expand_dims(img_array, axis=0)  # batch size = 1

# التنبؤ
preds = model.predict(img_array)
class_idx = np.argmax(preds[0])
confidence = np.max(preds[0])

print(f"📷 Image: {file_path}")
print(f"🔮 Predicted class: {class_names[class_idx]} ({confidence*100:.2f}%)")


Saving images (2).jpg to images (2) (2).jpg


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
📷 Image: images (2) (2).jpg
🔮 Predicted class: cat (99.88%)
